R Code

In [ ]:
#merge data
install.packages("readxl")
install.packages("dplyr")
install.packages("purrr")
install.packages("writexl")

library(readxl)
library(dplyr)
library(purrr)
library(writexl)

file_path <- "data_file_path"
excel_files <- list.files(path = file_path, pattern = "*.xlsx", full.names = TRUE)

data_list <- excel_files %>%
  map(~ read_excel(.))

merged_data <- reduce(data_list, ~ right_join(.x, .y, by = c("state", "category", "item_code", "item_name", "package_type")))
# merged_data <- reduce(data_list, ~ inner_join(.x, .y, by = "store_number"))

View(merged_data)
write_xlsx(merged_data, path = "data_file_path\\merged_tequila_data.xlsx")

map(data_list, colnames)

In [ ]:
#cleaning data
#data cleaning
# Loading
install.packages("writexl")
library(tidyverse)
library(lubridate)
library(randomForest)
library(caret)
library(tm)
library(text2vec)
library(SnowballC)
library(glmnet)
library(vip)
library(stringr)
library(dplyr)
library(ROCR)
library(ggplot2)
library(writexl)
#library(fastDummies)
library(gbm)
library(readxl)
# xlsx files
clean_data <- read_excel("merged_tequila_data.xlsx")
summary(clean_data)
#change to numeric
clean_data$TequilaUnder65 <- as.numeric(clean_data$TequilaUnder65)
clean_data$TequilaOver65 <- as.numeric(clean_data$TequilaOver65)
# Extract zipcode
clean_data <- clean_data %>%
  mutate(zipcode= str_extract(Store_Address, "\\d{5}$")) %>%
  mutate(zipcode= as.numeric(zipcode))%>%
  mutate(zipcode= as.factor(zipcode))
#standardize open_date
clean_data$Open_Date <- as.Date(clean_data$Open_Date, format = "%Y-%m-%d")
summary(clean_data$Open_Date)


# Define a mapping of tequila brand names to their categories
category_mapping <- c(
  "Blanco/Silver" = "Silver|Blanco",
  "Reposado" = "Reposado",
  "Anejo" = "Anejo",
  "Extra Anejo" = "Extra Anejo",
  "Mezcal" = "Mezcal",
  "Cristalino" = "Cristalino",
  "Gold"= "Gold")
# Create a new column for tequila category
clean_data <- clean_data %>%
  mutate(tequila_category = case_when(
    str_detect(item_name, category_mapping["Blanco/Silver"]) ~ "Blanco/Silver",
    str_detect(item_name, category_mapping["Reposado"]) ~ "Reposado",
    str_detect(item_name, category_mapping["Anejo"]) ~ "Anejo",
    str_detect(item_name, category_mapping["Extra Anejo"]) ~ "Extra Anejo",
    str_detect(item_name, category_mapping["Mezcal"]) ~ "Mezcal",
    str_detect(item_name, category_mapping["Cristalino"]) ~ "Cristalino",
    str_detect(item_name, category_mapping["Gold"]) ~ "Gold",
    TRUE ~ "Other" )) # Default category for unmatched brands  ))
additional_mapping <- c(
  "Jose Cuervo Especial Silv Tequila" = "Blanco/Silver",
  "Don Julio 1942 Tequila"= "Anejo",
  "Patron Tequila Gran Platinum" = "Blanco/Silver",
  "Hornitos Plata Tequila"= "Blanco/Silver",
  "Altos PlataTequila"= "Blanco/Silver",
  "Maestro Dobel Diamante"= "Reposado",
  "Herradura Ultra Tequila"= "Anejo",
  "Lobos 1707 Joven Tequila"= "Blanco/Silver",
  "400 Conejos Joven Oaxaca"= "Mezcal",
  "Gran Centenario Plata Tequila" = "Blanco/Silver",
  "Avion Reserva 44"= "Anejo",
  "Don Julio Alma Miel Joven Tequila"= "Cristalino",
  "Tequila Ocho Plata" = "Blanco/Silver",
  "Sauza Tres Generaciones Plata"="Blanco/Silver",
  "Corralejo 99000 Horas Tequila" = "Anejo",
  "Jose Cuervo Rsrva DeFamiliaPlatino" = "Blanco/Silver",
  "Montezuma White"= "Blanco/Silver",
  "Clase Azul Plata Tequila"= "Blanco/Silver",
  "Tarantula  Azul Tequila"= "Blanco/Silver",
  "Casa Dragones Tequila Joven"= "Blanco/Silver",
  "Don Fulano Fuerte Tequila"= "Blanco/Silver",
  "Gran Centenario Leyenda Extra Anej"= "Anejo",
  "Don Fulano Fuerte Tequila "= "Anejo",
  "Asombroso Platino Tequila"= "Blanco/Silver",
  "Herradura Seleccion SupremaTequila"= "Blanco/Silver",
  "San Matias Gran Reserva Tequila"= "Anejo",
  "Sauza Conmemorativo"= "Gold",
  "Durango White DSS"= "Blanco/Silver",
  "Republic Plata Texas Bottle"= "Blanco/Silver",
  "Del Majuey San Luis del Rio"= "Mezcal",
  "Don Julio Rosado"= "Reposado")

#Update the tequila_category in clean_data using additional_mapping
clean_data <- clean_data %>%
  mutate(tequila_category = case_when(
    item_name %in% names(additional_mapping) ~ additional_mapping[item_name],  # Assign from additional mapping
    TRUE ~ tequila_category  # Keep existing category if not in additional mapping
  ))
other_brands <- clean_data %>%
  filter(Sales_Bucket == "Sales excluded for model test") %>%
  select (item_name,tequila_category)%>%
  distinct() %>%
  filter(tequila_category== "Other")
other_brands_counts <- clean_data %>%
  filter(tequila_category == "Other") %>%
  group_by(item_name) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

#change to factor-all catergorical columns
clean_data <- clean_data %>%
  mutate(state = as.factor(state)) %>%
  mutate(package_type = as.factor(package_type))%>%
  mutate(item_name = as.factor(item_name))%>%
  mutate(Sales_Bucket = as.factor(Sales_Bucket))%>%
  mutate(PriceZone = as.factor(PriceZone))%>%
  mutate(item_code=as.factor(item_code))%>%
  mutate(store_number=as.factor(store_number))%>%
  mutate(Store_Size=as.factor(Store_Size))%>%
  mutate(tequila_category=as.factor(tequila_category))

# Function to calculate total ml for entries with "ml" and bottle count
calculate_total_ml <- function(package_type) {
  # Initialize total_ml as NA
  total_ml <- NA
  # Check if the entry matches the "quantity-bottle count gft" format (e.g., "375-3gft" or "50ml-4p")
  if (grepl("^[0-9]+ml?-[0-9]+[a-z]*$", package_type, ignore.case = TRUE)) {
    # Extract the ml quantity before "-"
    ml_quantity <- as.numeric(sub("ml?-.*", "", package_type))
    # Extract the number of units after "-"
    unit_count <- as.numeric(sub(".*-([0-9]+)[a-z]*$", "\\1", package_type))
    # Calculate total ml
    total_ml <- ml_quantity * unit_count
  } else if (grepl("^[0-9]+-[0-9]+gft$", package_type)) {
    # Extract the ml quantity before "-"
    ml_quantity <- as.numeric(sub("-.*", "", package_type))
    # Extract the bottle count after "-"
    bottle_count <- as.numeric(sub(".*-([0-9]+)gft", "\\1", package_type))
    # Calculate total ml
    total_ml <- ml_quantity * bottle_count
  } else if (grepl("^[0-9]+(\\.[0-9]+)?ml", package_type, ignore.case = TRUE)) {
    # Handle entries with "ml" followed by other text, like "mlBox"
    total_ml <- as.numeric(gsub("ml.*", "", package_type, ignore.case = TRUE))
  } else if (grepl("^[0-9]+(\\.[0-9]+)?L$", package_type, ignore.case = TRUE)) {
    # Handle entries with liters, convert to ml, allowing for decimal values
    total_ml <- as.numeric(gsub("L", "", package_type, ignore.case = TRUE)) * 1000
  } else if (grepl("^[0-9]+gft$", package_type, ignore.case = TRUE)) {
    # Handle entries like "375gft" as ml quantity
    total_ml <- as.numeric(gsub("gft", "", package_type, ignore.case = TRUE))
  } else {
    total_ml <- NA
  }

  return(total_ml)
}

# Apply the function to each row in the "package_type" column
clean_data$package_type_ml <- sapply(clean_data$package_type, calculate_total_ml)

#find price per 100ml
# Calculate price per 100 ml
calculate_price_per_100_ml <- function(package_type_ml, Retail) {
  if (package_type_ml > 0) {
    return((Retail / package_type_ml) * 100)
  } else {
    return(NA)  # Handle cases where ml is zero or missing
  }
}

#fill nas in retail
temp_data <- clean_data %>% filter(is.na(Retail))
clean_data$Retail[is.na(clean_data$Retail)] <- 18.99

# Apply this function to create a new column
clean_data$price_per_100_ml <- mapply(calculate_price_per_100_ml, clean_data$package_type_ml, clean_data$Retail)

# First, calculate sales_adjusted for each group based on item_code, and fill missing sales_dollars_L52wk values
clean_data <- clean_data %>%
  group_by(item_code) %>%
  mutate(
    normalized_households = Households / max(Households, na.rm = TRUE),  # Normalize within item_code group
    sales_adjusted = mean(sales_dollars_L52wk, na.rm = TRUE) * normalized_households
  ) %>%
  ungroup() %>%
  mutate(
    sales_dollars_L52wk = ifelse(is.na(sales_dollars_L52wk), sales_adjusted, sales_dollars_L52wk)
  )
# Next, calculate sales_adjusted by state and Retail, then fill missing sales values
clean_data <- clean_data %>%
  group_by(state, Retail) %>%
  mutate(
    normalized_households = Households / max(Households, na.rm = TRUE),  # Normalize within state and Retail group
    sales_adjusted2 = mean(sales_dollars_L52wk, na.rm = TRUE) * normalized_households
  ) %>%
  ungroup() %>%
  mutate(
    sales_dollars_L52wk = ifelse(is.na(sales_dollars_L52wk), sales_adjusted2, sales_dollars_L52wk)
  )
# Calculate sales_adjusted for Retail, and fill missing sales values
clean_data <- clean_data %>%
  group_by(Retail) %>%
  mutate(
    normalized_households = Households / max(Households, na.rm = TRUE),
    sales_adjusted3 = mean(sales_dollars_L52wk, na.rm = TRUE) * normalized_households
  ) %>%
  ungroup() %>%
  mutate(
    sales_dollars_L52wk = ifelse(is.na(sales_dollars_L52wk), sales_adjusted3, sales_dollars_L52wk)
  )
# Create Price Bands
clean_data <- clean_data %>%
  mutate(price_band = ifelse(Retail < 20, 1,
                             ifelse(Retail < 65, 2,
                                    ifelse(Retail < 150, 3, 4))),
         price_band = as.factor(price_band))
# Remaining NAs will be filled with the state, priceband
clean_data <- clean_data %>%
  group_by(state,price_band) %>%
  mutate(normalized_households = Households / max(Households, na.rm = TRUE),
         sales_adjusted4 = mean(sales_dollars_L52wk, na.rm = TRUE) * normalized_households
  ) %>%
  ungroup() %>%
  mutate(
    sales_dollars_L52wk = ifelse(is.na(sales_dollars_L52wk), sales_adjusted4, sales_dollars_L52wk)
  )
summary(clean_data$sales_dollars_L52wk)
#Calculate avg_pod. Use the overall median if all NAs for item code. Only when non-missing values exist, then normalize Households and calculate pod_adjusted.
overall_median_pod <- median(clean_data$points_of_distribution_L52wk, na.rm = TRUE)

clean_data <- clean_data %>%
  group_by(item_code) %>%
  mutate(
    avg_pod = ifelse(all(is.na(points_of_distribution_L52wk)), overall_median_pod, mean(points_of_distribution_L52wk, na.rm = TRUE)),,
    normalized_households = Households / max(Households),
    pod_adjusted = avg_pod * normalized_households
  ) %>%
  ungroup()
clean_data <- clean_data %>%
  mutate(
    points_of_distribution_L52wk = ifelse(is.na(points_of_distribution_L52wk), pod_adjusted, points_of_distribution_L52wk)
  )
summary(clean_data$points_of_distribution_L52wk)


#Filling NAs in L52w_in_Stock with 0 which doesn't have stock for that store
clean_data <- clean_data %>%
  mutate(
    L52W_in_Stock = ifelse(
      is.na(L52W_in_Stock) &
        ((item_code == "130634750" & store_number == "1008") |
           (item_code == "130634750" & store_number == "1010")),
      0,
      L52W_in_Stock
    )
  )

#Filling NAs in L52W_in_Stock with 52 which have stock in that store
clean_data <- clean_data %>%
  mutate(
    L52W_in_Stock = ifelse(
      is.na(L52W_in_Stock) &
        ((item_code == "130634750" & store_number == "1001") |
           (item_code == "149223375" & store_number == "1121")),
      52,
      L52W_in_Stock
    )
  )

#find the volatality score
clean_data <- clean_data %>%
  group_by(item_code) %>%
  mutate(
    mean_sales = mean(),
    sd_sales = ifelse(is.na(sd(sales_dollars_L52wk)),0,sd(sales_dollars_L52wk)),
    volatility_score = sd_sales / mean_sales
  )
# Apply K-means clustering to households data with 4 clusters
clean_data$households_cluster <- kmeans(clean_data$Households, centers = 4)$cluster
# Convert clusters to factors for coloring
clean_data$households_cluster <- as.factor(clean_data$households_cluster)
# Calculate mean households for each cluster and assign labels
cluster_means <- clean_data %>%
  group_by(households_cluster) %>%
  summarize(mean_households = mean(Households)) %>%
  arrange(mean_households) %>%
  mutate(density_label = c("Low", "Medium", "High", "Very High")) # Rename to density_label
# View the summary table
print(cluster_means)
# Join back to the original data with new labels
clean_data <- clean_data %>%
  left_join(cluster_means %>% select(households_cluster, density_label), by = "households_cluster") %>%
  mutate(households_density = factor(density_label, levels = c("Low", "Medium", "High", "Very High")))

#create a celebrity_owned column to check if the brand is owned by celebrity
celebrity_brands <- c("Casamigos", "818", "Teremana", "DeLeón", "Sauza", "Cabo Wabo", "Patron",
                      "Partida", "Ocho", "Herradura", "512", "El Tesoro", "Codigo 1530", "Dulce Vida",
                      "Avión", "Kosmos", "El Bandido Yankee", "Santo", "Lobos 1707", "Dos Primos", "Vida",
                      "Insolito", "Casa Del Sol", "Pantalones Organic", "Honor", "Gran Coramino",
                      "Dos Hombres", "Próspero", "Villa One", "JAJA", "Santo Fino", "Cincoro", "Flecha Azul")

# Function to check if an item is celebrity-owned
celebrity_owned <- function(item_name, brands) {
  any(sapply(brands, function(brand) grepl(brand, item_name, ignore.case = TRUE)))
}

# Apply the function to create a new column
clean_data$celebrity_owned <- as.integer(sapply(clean_data$item_name, celebrity_owned, brands = celebrity_brands))

#categorizing households
# Calculate the min and max values from the Households column
#min_households <- min(clean_data$Households, na.rm = TRUE)
#max_households <- max(clean_data$Households, na.rm = TRUE)

# Calculate the interval
#interval <- (max_households - min_households) / 4

#clean_data <- clean_data %>%
#  mutate(
#   household_density = case_when(
#      Households >= min_households & Households < (min_households + interval) ~ "low density",
#      Households >= (min_households + interval) & Households < (min_households + 2 * interval) ~ "med density",
#      Households >= (min_households + 2 * interval) & Households < (min_households + 3 * interval) ~ "high density",
#      Households >= (min_households + 3 * interval) ~ "very high density")
#  )

# Categorize Medium_HH_Income
clean_data <- clean_data %>%
  mutate( Income_Category = case_when(
    Median_HH_Income < 75000 ~ "Lower",
    Median_HH_Income >= 75000 & Median_HH_Income < 100000 ~ "Middle",
    Median_HH_Income >= 100000 & Median_HH_Income < 150000 ~ "Upper-Middle",
    Median_HH_Income >= 150000 ~ "High" ),
    Income_Category = factor(Income_Category, levels = c("Lower", "Middle", "Upper-Middle", "High")) )

# Create Sales Per Store Column
clean_data <- clean_data %>%
  mutate(sales_per_store = ifelse(points_of_distribution_L52wk == 0, 0 ,sales_dollars_L52wk/points_of_distribution_L52wk))
# Change date to years since opening
clean_data <- clean_data %>%
  mutate(
    years_open = as.numeric(difftime(Sys.Date(), Open_Date, units = "days")) / 365.25
  )

anejo_tequila_search <- read_csv("anejo_tequila_search.csv")
clean_data <- clean_data %>%
  left_join(anejo_tequila_search, by = "state")

google_trends <- read_csv("google_trends.csv")
clean_data <- clean_data %>%
  left_join(google_trends, by = "state")

reposado_tequila_search <- read_csv("reposado_tequila_search.csv")
clean_data <- clean_data %>%
  left_join(reposado_tequila_search, by = "state")

silver_tequila_search <- read_csv("silver_tequila_search.csv")
clean_data <- clean_data %>%
  left_join(silver_tequila_search, by = "state")

cheap_tequila_search <- read_csv("cheap_tequila_search.csv")
clean_data <- clean_data %>%
  left_join(cheap_tequila_search, by = "state")

best_tequila_search <- read_csv("best_tequila_search.csv")
clean_data <- clean_data %>%
  left_join(best_tequila_search, by = "state")

clean_data <- clean_data %>%
  mutate(state = as.factor(state))

#DOWNLOAD EXCEL FOR EDA
#write_xlsx(clean_data, "clean_data.xlsx")
# This will save the file in your working directory. To check the path:
#getwd()


#removing helper columns and non-model columns
clean_data <- clean_data %>%
  select(-category,-vintage,-State,-Store_Name,-Store_Address,-sales_adjusted,-package_type_ml,-pod_adjusted,
         -avg_pod,-mean_sales,-sd_sales,-households_cluster,-density_label,-Households, -sales_adjusted2,
         -sales_adjusted3,-sales_adjusted4, -zipcode, -Open_Date,-item_name)
summary(clean_data)

# Validation
sapply(clean_data, class)
# All are numeric, factor, or date

#split the data into train and test
train<- clean_data %>% filter(Sales_Bucket == "Sales included")
test<- clean_data %>% filter(Sales_Bucket == "Sales excluded for model test")

anyNA(train)
na_rows <- train[!complete.cases(train), ]

In [ ]:
#EDA Code
# Plot box plot to see households categories
ggplot(clean_data, aes(x = households_density, y = Households, fill = households_density)) +
  geom_boxplot(alpha = 0.7) +
  labs(title = "Households Distribution by Density Level", x = "Density Level", y = "Number of Households") +
  scale_fill_manual(values = c("Low" = "#1b9e77", "Medium" = "#d95f02", "High" = "#7570b3", "Very High" = "#e7298a")) +
  theme_minimal()

#bar grapgh to see mean sales by state by tequila category
ggplot(sales_summary, aes(x = state, y = mean_normalized_sales, fill = tequila_category)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Mean Normalized Sales by Tequila Category and State",
       x = "State",
       y = "Mean Normalized Sales (L52W)") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

SyntaxError: invalid syntax (<ipython-input-1-251e9158557f>, line 3)

# Initial Modeling

In [ ]:
#split the data into full dataset and final test for submission.
full<- clean_data %>% filter(Sales_Bucket == "Sales included")
final_test<- clean_data %>% filter(Sales_Bucket == "Sales excluded for model test")

# Split into basic train and test
set.seed(1)
train_indices <- sample(seq_len(nrow(full)), size = 0.8 * nrow(full))

# Split the data
train <- full[train_indices, ]
test <- full[-train_indices, ]

###############################################################################

##########
### Baseline Model RMSE

baseline_prediction <- rep(mean(labels_test, na.rm = TRUE), length(labels_test))
baseline_rmse <- RMSE(baseline_prediction, labels_test)
baseline_MAE <- MAE(baseline_prediction, labels_test)
print(baseline_rmse)
print(baseline_MAE)

##########
### Random Forest does not work as is. Too many item codes.

# model <- randomForest(Normalized_Sales_L52W ~ ., data = full, ntree = 500)


##########
### Boosting does not work because we need to convert to integer/numeric for each.
### Doing so loses a lot of information

#dtrain <- xgb.DMatrix(data = as.matrix(full[, -which(names(full) == "Normalized_Sales_L52W")]),
#                     label = full$Normalized_Sales_L52W)
#model <- xgb.train(params = list(objective = "reg:squarederror"), data = dtrain, nrounds = 100)

##########
### Catboost does not work because there is not a catboost regression module in R.
### Might consider doing in python.

# Run these to download catboost. If/When prompted, update all by entering 1 into the console.
#install.packages('remotes')
#remotes::install_url('https://github.com/catboost/catboost/releases/download/v1.2.7/catboost-R-windows-x86_64-1.2.7.tgz', INSTALL_opts = c("--no-multiarch", "--no-test-load"))

# Quickstart guide: https://catboost.ai/en/docs/concepts/r-quickstart

library(catboost)

# Separate into features and labels
features <- train %>% select(-Normalized_Sales_L52W)
labels <- train$Normalized_Sales_L52W

features_test <- test %>% select(-Normalized_Sales_L52W)
labels_test <- test$Normalized_Sales_L52W

train_pool <- catboost.load_pool(data = features, label = labels)

model <- catboost.train(train_pool,  NULL,
                        params = list(loss_function = 'RMSE',
                                      iterations = 100, metric_period=10))
model_MAE <- catboost.train(train_pool, NULL,
                        params = list(loss_function = 'MAE',
                                      iterations = 100, metric_period = 10))



test_pool <- catboost.load_pool(features_test)

prediction <- catboost.predict(model, test_pool)
prediction_MAE <- catboost.predict(model_MAE, test_pool)

catboost_rmse <- RMSE(prediction,labels_test)
catboost_MAE <- MAE(prediction_MAE, labels_test)

print(baseline_rmse)
print(catboost_rmse)

print(baseline_MAE)
print(catboost_MAE)

baseline_error <- .5*baseline_rmse + .5*baseline_MAE
catboost_error <- .5*catboost_rmse + .5*catboost_MAE

print(paste("Baseline Error: ",  baseline_error))
print(paste("Catboost Error: ",  catboost_error))

#> print(baseline_MAE)
#[1] 4186.446
#> print(baseline_rmse)
#[1] 10024.44

#> print(baseline_MAE)
#[1] 4186.446
#> print(catboost_MAE)
#[1] 2174.655

# [1] "Baseline Error:  7105.44308138542"
# [1] "Catboost Error:  3310.07157977977"


In [ ]:
###############################################################################
## Cross Validation and Grid Search

# Split into basic train and test
set.seed(1)
train_indices <- sample(seq_len(nrow(full)), size = 0.9 * nrow(full))

# Split the data
train <- full[train_indices, ]
holdout <- full[-train_indices, ]

# Separate into train/validation and holdout set.Uncomment if not doing gridsearch.

#features <- train %>% select(-Normalized_Sales_L52W)
#labels <- train$Normalized_Sales_L52W

#features_holdout <- holdout %>% select(-Normalized_Sales_L52W)
#labels_holdout <- holdout$Normalized_Sales_L52W



grid_search <- function(){

  learning_rate = c(0.01, 0.03, 0.1)
  depth = c(4, 6, 8)
  l2_leaf_reg = c(1, 3)
  iterations = c(500)

  # Change the second number to be equal to numfolds.
  foldnumber <- seq(1,5,1)

  numfolds <- 2

  train_shuffled <- train[sample(nrow(train)),]
  train_x <- train_shuffled %>% select(-Normalized_Sales_L52W)
  train_y <- train_shuffled$Normalized_Sales_L52W


  #separate data into k equally-sized folds
  #cut() will assign "fold numbers" to each instance
  folds <- cut(seq(1,nrow(train_shuffled)),breaks=numfolds,labels=FALSE)

  # Initialize an empty dataframe to store results
  results <- data.frame(fold = numeric(),
                        learning_rate = numeric(),
                        depth = numeric(),
                        l2_leaf_reg = numeric(),
                        RMSE = numeric(),
                        MAE = numeric(),
                        Error = numeric(),
                        stringsAsFactors = FALSE)


  #nested loops to tune these three parameters
  print('fold, learning_rate, depth, l2_leaf_reg, iterations, RMSE, MAE, Error')
  for(f in c(1:length(unique(folds)))){
    for(i in c(1:length(learning_rate))){
      for(j in c(1:length(depth))){
        for(k in c(1:length(l2_leaf_reg))){
          for (l in c(1:length(iterations))){

            set.seed(sample(1:100000,1,replace=TRUE))

            thisfold <- foldnumber[f]
            thislearning_rate <- learning_rate[i]
            thisdepth <- depth[j]
            thisl2_leaf_reg <- l2_leaf_reg[k]
            thisiteration <- iterations[l]

            valid_inds <- which(folds==f,arr.ind=TRUE)
            valid_fold <- train_x[valid_inds, ]
            valid_fold_y <- train_y[valid_inds]
            train_fold <- train_x[-valid_inds, ]
            train_fold_y <- train_y[-valid_inds]

            train_data <- catboost.load_pool(data = train_fold,
                                             label = train_fold_y)
            valid_data <- catboost.load_pool(data = valid_fold,
                                             label = valid_fold_y)

            model_RMSE <- catboost.train(
              learn_pool = train_data,
              test_pool = valid_data,
              params = list(
                loss_function = "RMSE",  # Adjust for regression tasks
                learning_rate = thislearning_rate,
                depth = thisdepth,
                l2_leaf_reg = thisl2_leaf_reg,
                iterations = thisiteration,
                eval_metric = "RMSE",
                use_best_model = TRUE
              )
            )

            model_MAE <- catboost.train(
              learn_pool = train_data,
              test_pool = valid_data,
              params = list(
                loss_function = "MAE",  # Adjust for regression tasks
                learning_rate = thislearning_rate,
                depth = thisdepth,
                l2_leaf_reg = thisl2_leaf_reg,
                iterations = thisiteration,
                eval_metric = "MAE",
                use_best_model = TRUE
              )
            )

            prediction_RMSE <- catboost.predict(model_RMSE, valid_data)
            prediction_MAE <- catboost.predict(model_MAE, valid_data)

            catboost_rmse <- RMSE(prediction_RMSE,valid_fold_y)
            catboost_MAE <- MAE(prediction_MAE, valid_fold_y)

            catboost_error <- .5*catboost_rmse + .5*catboost_MAE


            #print the performance for every combination
            print(paste(thisfold, thislearning_rate, thisdepth, thisl2_leaf_reg, thisiteration, catboost_rmse, catboost_MAE, catboost_error,sep = ", "))
            # Append the results to the dataframe
            results <- rbind(results, data.frame(fold = thisfold,
                                                 learning_rate = thislearning_rate,
                                                 depth = thisdepth,
                                                 l2_leaf_reg = thisl2_leaf_reg,
                                                 iterations = thisiteration,
                                                 RMSE = catboost_rmse,
                                                 MAE = catboost_MAE,
                                                 Error = catboost_error))

          }
        }
      }
    }
  }
  return(results)
}

result <- grid_search()


In [ ]:
###############################################################################
###### Determining Best Model Parameters from the Grid Search

fold_results <- result %>% group_by(learning_rate, depth, l2_leaf_reg, iterations) %>% summarise(
  mean_RMSE = mean(RMSE),
  mean_MAE = mean(MAE),
  mean_error = mean(Error))

best_error <- min(fold_results$mean_error)

best_params <- fold_results %>% filter(mean_error == best_error)

################################################################################
###### Training Best Model on Final Validation Set

train_data <- catboost.load_pool(data = features,
                                 label = labels)
valid_data <- catboost.load_pool(data = features_test,
                                 label = labels_test)

model_RMSE <- catboost.train(
  learn_pool = train_data,
  test_pool = valid_data,
  params = list(
    loss_function = "RMSE",  # Adjust for regression tasks
    learning_rate = best_params$learning_rate,
    depth = best_params$depth,
    l2_leaf_reg = best_params$l2_leaf_reg,
    iterations = best_params$iterations,
    eval_metric = "RMSE",
    use_best_model = TRUE
  )
)

model_MAE <- catboost.train(
  learn_pool = train_data,
  test_pool = valid_data,
  params = list(
    loss_function = "RMSE",  # Adjust for regression tasks
    learning_rate = best_params$learning_rate,
    depth = best_params$depth,
    l2_leaf_reg = best_params$l2_leaf_reg,
    iterations = best_params$iterations,
    eval_metric = "MAE",
    use_best_model = TRUE
  )
)


### Making Predictions
prediction_RMSE <- catboost.predict(model_RMSE, valid_data)
prediction_MAE <- catboost.predict(model_MAE, valid_data)

### Final Prediction
prediction_final <- .5*(prediction_RMSE + prediction_MAE)

### Merging Final Predictions with Validation Data
prediction_final <- as.vector(prediction_final)
test_with_predictions <- cbind(test, prediction_final = prediction_final)

### Merging Train and Validation Data
train$prediction_final <- NA
full_df <- rbind(train,test_with_predictions)


### Calculating Residuals
full_df <- full_df %>% mutate(residuals = prediction_final - Normalized_Sales_L52W )

### Calculating Final Error
final_RMSE <- RMSE(test_with_predictions$prediction_final,test_with_predictions$Normalized_Sales_L52W)
final_MAE <- MAE(test_with_predictions$prediction_final,test_with_predictions$Normalized_Sales_L52W)
final_error <- .5*final_RMSE + .5*final_MAE

### Final Error is 2613.9


### Writing to xlsx
write_xlsx(full_df, "full_df.xlsx")

In [ ]:
###feature engineering for over and under performing grouped by store


full_residuals_data <- read_excel("clean_data.xlsx")
summary(full_residuals_data)
full_residuals_data <- full_residuals_data %>%
  group_by(store_number) %>%
  mutate(
    mean_residual = mean(residuals),   # Mean residual for the store
    sd_residual = sd(residuals),       # Standard deviation of residuals for the store
    lower_threshold = mean_residual - 2* sd_residual,  # Lower threshold (Mean - 2 * SD)
    upper_threshold = mean_residual + 2* sd_residual   # Upper threshold (Mean + 2 * SD)
  ) %>%
  ungroup()  # Remove grouping after calculation

# Flag underperforming or overperforming stores based on thresholds
full_residuals_data <- full_residuals_data %>%
  mutate(
    underperforming = ifelse(residuals < lower_threshold, 1, 0),
    overperforming = ifelse(residuals > upper_threshold, 1, 0)
  )
write_xlsx(full_residuals_data, "full_residuals_data.xlsx")